In [ ]:
import json

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [ ]:
data = [json.loads(line) for line in open('Datasets/AGNews/data/test.jsonl')]

In [ ]:
rationales = [json.loads(line) for line in open('outputs/bert_classification/AGNews/AGNews_base_bert_wrapper_min/integrated_gradient_rationales/test.jsonl')]

In [ ]:
lengths_d = np.array([len(x['document'].split()) for x in data[:len(rationales)]])
lengths_r = np.array([len(x['document'].split()) for x in rationales])

In [ ]:
sns.distplot(lengths_r/lengths_d, bins=30)
np.mean(lengths_r/lengths_d)

In [ ]:
np.std(lengths_r/lengths_d)

In [2]:
from Rationale_Analysis.models.saliency_scorer import WrapperSaliency
from Rationale_Analysis.models.classifiers import BERTRationaleModel